<p>Author: Wesley Schonborn
<p>Date: 4/27/2021
<p>Project: Port Saint Lucie Neighborhoods...Which Ones Need a Thai Restaurant?

<h1>Importing</h1>

In [ ]:
import numpy as np
import requests # library to handle requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Install and import geopy
!conda install -c conda-forge geopy --yes # only need to run once
from geopy.geocoders import Nominatim

# install and import folium for mapping
!conda install -c conda-forge folium=0.5.0 --yes # only needed to run once
import folium

<h1>Import neighborhood data to a dataframe</h1>

In [ ]:
# Put csv of neighborhoods into "neighborhoods" variable
neighborhoods = pd.read_csv("psl_neighborhoods.csv")

# Quick look at the dataframe
print(neighborhoods.head())
print(neighborhoods.shape)

<h1>Functions</h1>

In [ ]:
# Function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=2500, categoryId='4d4b7105d754a06374d81259'):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT,
            categoryId)  # categoryId to limit to restaurants/food places
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']
    return(nearby_venues)

# Function to get most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

<h1>Map Setup</h1>

In [ ]:
address = 'Port Saint Lucie, FL'  # Should be something that is in the database for geolocator
geolocator = Nominatim(user_agent="psl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Port Saint Lucie, FL are {}, {}.'.format(latitude, longitude))
# create map
map_psl = folium.Map(location=[latitude, longitude], zoom_start=12)

<h1>Add markers to map</h1>

In [ ]:
for lat, lng, borough, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['borough'], neighborhoods['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_psl)

# display map (in Jupyter)
map_psl

<h1>Foursquare login info</h1>

In [ ]:
CLIENT_ID = 'PZ1XLTP2AE3UZCNEFM5ZVGAEIITCYBIWITPAZNTUAWMIF43C'  # your Foursquare ID
CLIENT_SECRET = 'LOOLAGGD4WSUF42S4OVATNV1DGI1FERGONOGIQYWCZNE0FLZ'  # your Foursquare Secret
VERSION = '20180605'  # Foursquare API version
LIMIT = 100  # A default Foursquare API limit value

# Print out to make sure credentials are correct
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

<h1>Get venues and show neighborhoods</h1>

In [ ]:
psl_venues = getNearbyVenues(names = neighborhoods['neighborhood'], latitudes = neighborhoods['latitude'], longitudes = neighborhoods['longitude'])

In [ ]:
print(psl_venues.shape)

psl_venues.groupby('Neighborhood').count()
print('There are {} unique categories.'.format(len(psl_venues['Venue Category'].unique())))

In [ ]:
psl_venues

<h1>One hot encoding</h1>

In [ ]:
psl_onehot = pd.get_dummies(psl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
psl_onehot['Neighborhood'] = psl_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [psl_onehot.columns[-1]] + list(psl_onehot.columns[:-1])
psl_onehot = psl_onehot[fixed_columns]

psl_onehot.head()

<h1>Show all neighborhoods and how many Thai restaurants each has.</h1>

In [ ]:
thai = pd.DataFrame()
thai = psl_onehot.groupby(by=['Neighborhood'])['Thai Restaurant'].sum()
thai = thai.to_frame() # Python converts to series for some reason
thai

<h1>Shows the neighborhoods without Thai restaurants.</h1>

In [ ]:
no_thai = thai[thai['Thai Restaurant'] == 0]
no_thai

<h1>Join population data to the neighborhoods without Thai restaurants.</h1>

In [ ]:
population = neighborhoods[['neighborhood', 'population']]
no_thai_pop = pd.DataFrame()
no_thai_pop = no_thai.merge(population, how = "left", left_on = 'Neighborhood', right_on = 'neighborhood')
no_thai_pop = no_thai_pop.sort_values(by = 'population', ascending = False)
no_thai_pop

<h1>Map of neighborhoods with or without Thai restaurants</h1>
<h3>Red = no Thai restaurant</h3>
<h3>Green = Thai restaurant</h3>
<h3>Size of marker is relative to population</h3>

In [ ]:
neighborhoods_thai_no_thai = neighborhoods.merge(thai, how = "left", left_on = 'neighborhood', right_on = 'Neighborhood')

map_psl = folium.Map(location=[latitude, longitude], zoom_start=12) # Sometimes the map doesn't refresh.
for lat, lng, neighborhood, thai_rest, pop in zip(neighborhoods_thai_no_thai['latitude'], neighborhoods_thai_no_thai['longitude'], neighborhoods_thai_no_thai['neighborhood'], neighborhoods_thai_no_thai['Thai Restaurant'], neighborhoods_thai_no_thai['population']):
    if (thai_rest == 1):
        label = '{}, {}'.format(neighborhood, pop)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=pop/800,
            popup=label,
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=0.7,
            parse_html=False).add_to(map_psl)
    elif (thai_rest == 0):
        label = '{}, {}'.format(neighborhood, pop)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=pop/800,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_psl)
        
map_psl